# 1. 딥러닝 학습방법 - 순전파( Forward Propagation )

## 1.1 행렬의 곱셈을 이용한 순전파(Forward Propagation)

![Alt text]( neuralnetwork_final.png )

- 위와 같은 인공 신경망이 있다고 합시다. 주어진 인공 신경망을 케라스로 구현해본다면 아래와 같이 짧은 코드로 구현할 수 있습니다.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential() # 층을 추가할 준비
model.add( Dense( 8, input_dim = 4, init = 'uniform', activation = 'relu' ) )
# 입력층(4)과 다음 은닉층(8) 그리고 은닉층의 활성화 함수는 relu
model.add( Dense( 8, activation = 'relu' ) ) # 은닉층(8)의 활성화 함수는 relu
model.add( Dense( 3, activation = 'softmax' ) ) # 출력층(3)의 활성화 함수는 softmax

- 인공 신경망에서 입력층에서 출력층 방향으로 연산을 진행하는 과정을 순전파(Forward Propagation)라고 합니다. 
- 다르게 말하면 주어진 입력으로부터 예측값을 계산하는 과정을 순전파라고 합니다. 
- 벡터와 행렬 연산이 순전파 과정에서 층(layer)마다 적용이 됩니다.
- 케라스로 인공 신경망을 만들면 이러한 연산 과정을 자세히 이해하지 않아도 모델을 만들 수는 있지만, Numpy 등을 통해 인공 신경망을 로우 레벨로 개발하고 있다면 인공 신경망 내부 연산에 사용하는 행렬 크기를 고려해야 인공 신경망을 구현할 수 있습니다

### 1) layer 1의 행렬 크기 추정하기

- 우선 각 층을 기준으로 입력과 출력의 개수를 정리하면 다음과 같습니다.

    입력층 : 4개의 입력과 8개의 출력  
    은닉층1 : 8개의 입력과 8개의 출력  
    은닉층2 : 8개의 입력과 3개의 출력  
    출력층 : 3개의 입력과 3개의 출력  
  
  
- 편의상 입력층을 layer 0, 은닉층 1을 layer 1, 은닉층 2를 layer 2, 출력층을 layer 3라고 해봅시다. 
- 이제 위의 정보를 가지고 층마다 생기는 가중치와 편향 행렬의 크기를 추정해봅시다. 
- 가중치 행렬에 입력 행렬을 곱하는 경우와 입력 행렬에 가중치 행렬을 곱하는 경우가 있겠으나, 여기서는 후자를 가정합니다. 
- 또한 배치 크기는 1로 합니다. 이 경우 layer 1에서 처음 입력으로 들어오는 입력 행렬 X의 크기는 1 × 4로 행벡터에 해당됩니다. (만약 미니 배치 학습을 가정할 경우, X의 크기는 배치의 크기 × 4가 됩니다.)

- 입력 행렬, 가중치 행렬, 편향 행렬, 출력 행렬은 다음과 같은 크기 관계를 가집니다.

\begin{equation*}
    X_{m\ \text{×}\ n} × W_{n\ \text{×}\ j} + B_{m\ \text{×}\ j} = Y_{m\ \text{×}\ j}
\end{equation*}

- layer 1의 입력 행렬 X 의 크기는 1 × 4입니다. layer 1의 출력은 8개이므로, 그에 따라 출력 행렬 Y의 크기는 1 × 8이 됩니다.

\begin{equation*}
    X_{1\ \text{×}\ 4} × W_{n\ \text{×}\ j} + B_{m\ \text{×}\ j} = Y_{1\ \text{×}\ 8}
\end{equation*}

- 그런데 가중치 행렬 W의 행은 입력 행렬 X의 열과 같아야 하므로 아래와 같습니다.

\begin{equation*}
    X_{1\ \text{×}\ 4} × W_{4\ \text{×}\ j} + B_{m\ \text{×}\ j} = Y_{1\ \text{×}\ 8}
\end{equation*}

- 편향 행렬 B는 출력 행렬 Y의 크기에 영향을 주지 않으므로 편향 행렬 B의 크기는 출력 행렬 Y의 크기와 같습니다.

\begin{equation*}
    X_{1\ \text{×}\ 4} × W_{4\ \text{×}\ j} + B_{1\ \text{×}\ 8} = Y_{1\ \text{×}\ 8}
\end{equation*}

- 가중치 행렬 W의 열은 출력 행렬 Y의 열과 동일해야 합니다.

\begin{equation*}
    X_{1\ \text{×}\ 4} × W_{4\ \text{×}\ 8} + B_{1\ \text{×}\ 8} = Y_{1\ \text{×}\ 8}
\end{equation*}

- layer 1의 가중치 행렬과 편향 행렬의 크기를 구했습니다. 이제 layer 1의 출력 행렬 Y는 layer 2에서는 입력 행렬 X가 됩니다.

### 1) layer 2와 layer 3의 행렬 크기 추정하기

- 이를 반복하면 layer 2와 layer 3에서의 가중치 행렬과 편향 행렬의 크기를 구할 수 있습니다. 비록 은닉층과 출력층에 활성화 함수가 존재하지만 활성화 함수는 행렬의 크기에 영향을 주지 않습니다.

\begin{equation*}
    layer 2 : X_{1\ \text{×}\ 8} × W_{8\ \text{×}\ 8} + B_{1\ \text{×}\ 8} = Y_{1\ \text{×}\ 8}  
\end{equation*}
\begin{equation*} 
    layer 3 : X_{1\ \text{×}\ 8} × W_{8\ \text{×}\ 3} + B_{1\ \text{×}\ 3} = Y_{1\ \text{×}\ 3}
\end{equation*}
- 인공 신경망이 입력층에서 은닉층을 지나 출력층에서 예측값을 계산하기까지의 과정을 행렬 연산으로 가정하고 행렬의 크기를 추정해보았습니다. 
- 순전파를 진행하고 예측값을 구하고나서 이 다음에 인공 신경망이 해야할 일은 예측값과 실제값으로부터 오차를 계산하고, 오차로부터 가중치와 편향을 업데이트하는 일입니다. 즉, 인공 신경망의 학습 단계에 해당됩니다. 
- 이때 인공 신경망은 순전파와는 반대 방향으로 연산을 진행하며 가중치를 업데이트하는데, 이 과정을 역전파(BackPropagation)라고 합니다. 

## 1.2 순전파(Foward Propagation)

![Alt text]( forward_propagation.png )

- 활성화 함수, 은닉층의 수, 각 은닉층의 뉴런 수 등 딥 러닝 모델을 설계하고나면 입력값은 입력층, 은닉층을 지나면서 각 층에서의 가중치와 함께 연산되며 출력층으로 향합니다. 그리고 출력층에서 모든 연산을 마친 예측값이 나오게 됩니다. 이와 같이 입력층에서 출력층 방향으로 예측값의 연산이 진행되는 과정을 순전파라고 합니다.

## 1.3 손실 함수(Loss function)

![Alt text]( forward_loss_function.png )

- 손실 함수는 실제값과 예측값의 차이를 수치화해주는 함수입니다. 이 두 값의 차이. 즉, 오차가 클 수록 손실 함수의 값은 크고 오차가 작을 수록 손실 함수의 값은 작아집니다. 
- 회귀에서는 평균 제곱 오차, 분류 문제에서는 크로스 엔트로피를 주로 손실 함수로 사용합니다. 
- 손실 함수의 값을 최소화하는 두 개의 매개변수인 가중치 W와 편향 b를 찾아가는 것이 딥 러닝의 학습 과정이므로 손실 함수의 선정은 매우 중요합니다.

### 1) MSE(Mean Squared Error, MSE)

- 오차 제곱 평균을 의미합니다. 연속형 변수를 예측할 때 사용됩니다.

\begin{equation*}
    -\frac{1}{N} \sum (y - \hat{y})^2
\end{equation*}

### 2) 크로스 엔트로피(Cross-Entropy)

- y : 실제값 (0 or 1) / ŷ  : 예측값 (확률)

\begin{equation*}
    -\sum ylog\hat{y}
\end{equation*}

- 낮은 확률로 예측해서 맞추거나, 높은 확률로 예측해서 틀리는 경우 loss가 더 큽니다. 이진 분류 (Binary Classification)의 경우 binary_crossentropy를 사용하며 다중 클래스 분류(Multi-Class Classification)일 경우 categorical_crossentropy를 사용합니다.

## 1.4 옵티마이저(Optimizer)

![Alt text]( optimizer.png )

- 손실 함수의 값을 줄여나가면서 학습하는 방법은 어떤 옵티마이저를 사용하느냐에 따라 달라집니다. 
- 여기서 배치(Batch)라는 개념에 대한 이해가 필요합니다. 배치는 가중치 등의 매개 변수의 값을 조정하기 위해 사용하는 데이터의 양을 말합니다. 전체 데이터를 가지고 매개 변수의 값을 조정할 수도 있고, 정해준 양의 데이터만 가지고도 매개 변수의 값을 조정할 수 있습니다.

### 1) 배치 경사 하강법(Batch Gradient Descent)

- 배치 경사 하강법(Batch Gradient Descent)은 가장 기본적인 경사 하강법입니다. 배치 경사 하강법은 옵티마이저 중 하나로 오차(loss)를 구할 때 전체 데이터를 고려합니다. 머신 러닝에서는 1번의 훈련 횟수를 1 에포크라고 하는데, 배치 경사 하강법은 한 번의 에포크에 모든 매개변수 업데이트를 단 한 번 수행합니다. 배치 경사 하강법은 전체 데이터를 고려해서 학습하므로 에포크당 시간이 오래 걸리며, 메모리를 크게 요구한다는 단점이 있으나 글로벌 미니멈을 찾을 수 있다는 장점이 있습니다.

model.fit(X_train, y_train, batch_size=len(trainX))

### 확률적 경사 하강법(Stochastic Gradient Descent, SGD)

- 기존의 배치 경사 하강법은 전체 데이터에 대해서 계산을 하다보니 시간이 너무 오래걸린다는 단점이 있습니다. 확률적 경사 하강법은 매개변수 값을 조정 시 전체 데이터가 아니라 랜덤으로 선택한 하나의 데이터에 대해서만 계산하는 방법입니다. 더 적은 데이터를 사용하므로 더 빠르게 계산할 수 있습니다.

![Alt text]( SGD.png )

- 매개변수의 변경폭이 불안정하고, 때로는 배치 경사 하강법보다 정확도가 낮을 수도 있지만 속도만큼은 배치 경사 하강법보다 빠르다는 장점이 있습니다. 케라스에서는 아래와 같이 사용합니다.

model.fit(X_train, y_train, batch_size=1)

### 3) 미니 배치 경사 하강법(Mini-Batch Gradient Descent)

- 전체 데이터도 아니고, 1개의 데이터도 아니고 정해진 양에 대해서만 계산하여 매개 변수의 값을 조정하는 경사 하강법을 미니 배치 경사 하강법이라고 합니다. 미니 배치 경사 하강법은 전체 데이터를 계산하는 것보다 빠르며, SGD보다 안정적이라는 장점이 있습니다. 실제로 가장 많이 사용되는 경사 하강법입니다.

model.fit(X_train, y_train, batch_size=32) #32를 배치 크기로 하였을 경우

### 4) 모멘텀(Momentum)

- 모멘텀(Momentum)은 관성이라는 물리학의 법칙을 응용한 방법입니다. 모멘텀 SGD는 경사 하강법에 관성을 더 해줍니다. 모멘텀은 SGD에서 계산된 접선의 기울기에 한 시점(step) 전의 접선의 기울기값을 일정한 비율만큼 반영합니다. 이렇게 하면 마치 언덕에서 공이 내려올 때, 중간에 작은 웅덩이에 빠지더라도 관성의 힘으로 넘어서는 효과를 줄 수 있습니다.

![Alt text]( localmimum.png )

- 다시 말해 로컬 미니멈에 도달하였을 때, 기울기가 0이라서 기존의 경사 하강법이라면 이를 글로벌 미니멈으로 잘못 인식하여 계산이 끝났을 상황이라도 모멘텀. 즉, 관성의 힘을 빌리면 값이 조절되면서 로컬 미니멈에서 탈출하는 효과를 얻을 수도 있습니다. 케라스에서는 다음과 같이 사용합니다.

keras.optimizers.SGD(lr = 0.01, momentum= 0.9)

### 5) 아다그라드(Adagrad)

- 매개변수들은 각자 의미하는 바가 다른데, 모든 매개변수에 동일한 학습률(learning rate)을 적용하는 것은 비효율적입니다. 아다그라드는 각 매개변수에 서로 다른 학습률을 적용시킵니다. 이 때, 변화가 많은 매개변수는 학습률이 작게 설정되고 변화가 적은 매개변수는 학습률을 높게 설정시킵니다. 케라스에서는 다음과 같이 사용합니다.

keras.optimizers.Adagrad(lr=0.01, epsilon=1e-6)

### 6) 알엠에스프롭(RMSprop)

- 아다그라드는 학습을 계속 진행한 경우에는, 나중에 가서는 학습률이 지나치게 떨어진다는 단점이 있는데 이를 다른 수식으로 대체하여 이러한 단점을 개선하였습니다. 케라스에서는 다음과 같이 사용합니다.

keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06)

### 7) 아담(Adam)

- 아담은 알엠에스프롭과 모멘텀 두 가지를 합친 듯한 방법으로, 방향과 학습률 두 가지를 모두 잡기 위한 방법입니다. 케라스에서는 다음과 같이 사용합니다.

keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

케라스의 옵티마이저 사용법은 아래의 링크에서 좀 더 상세히 확인할 수 있습니다.  
링크 : https://keras.io/optimizers/

# 2. 딥러닝 학습방법 - 역전파( Back Propagation )

- 인공 신경망이 순전파 과정을 진행하여 예측값과 실제값의 오차를 계산하였을 때 어떻게 역전파 과정에서 경사 하강법을 사용하여 가중치를 업데이트하는지 직접 계산을 통해 이해해봅시다.

## 1. 인공 신경망의 이해(Neural Network Overview)

- 우선 예제를 위해 사용될 인공 신경망을 소개합니다. 
- 역전파의 이해를 위해서 여기서 사용할 인공 신경망은 입력층, 은닉층, 출력층 이렇게 3개의 층을 가집니다. 
- 또한 해당 인공 신경망은 두 개의 입력과, 두 개의 은닉층 뉴런, 두 개의 출력층 뉴런을 사용합니다. 
- 은닉층과 출력층의 모든 뉴런은 활성화 함수로 시그모이드 함수를 사용합니다.

![Alt text]( nn1_final.png )

- 위의 그림은 여기서 사용할 인공 신경망의 모습을 보여줍니다. 
- 은닉층과 출력층의 모든 뉴런에서 변수 z가 존재하는데 여기서 변수 z는 이전층의 모든 입력이 각각의 가중치와 곱해진 값들이 모두 더해진 가중합을 의미합니다. 
- 이 값은 뉴런에서 아직 시그모이드 함수를 거치지 않은 상태입니다. 즉, 활성화 함수의 입력을 의미합니다. 
- z 우측의 |를 지나서 존재하는 변수 h 또는 o는 z가 시그모이드 함수를 지난 후의 값으로 각 뉴런의 출력값을 의미합니다. 
- 이번 역전파 예제에서는 인공 신경망에 존재하는 모든 가중치 W에 대해서 역전파를 통해 업데이트하는 것을 목표로합니다. 
- 해당 인공 신경망은 편향 b는 고려하지 않습니다.

## 2. 순전파(Forward Propagation)

![Alt text]( nn2_final_final.png )

- 주어진 값이 위의 그림과 같을 때 순전파를 진행해봅시다. 
- 위의 그림에서 소수점 앞의 0은 생략하였습니다. 예를 들어 .25는 0.25를 의미합니다. 
- 파란색 숫자는 입력값을 의미하며, 빨간색 숫자는 각 가중치의 값을 의미합니다. 
- 앞으로 진행하는 계산의 결과값은 소수점 아래 여덟번째 자리까지 반올림하여 표기합니다.


- 각 입력은 입력층에서 은닉층 방향으로 향하면서 각 입력에 해당하는 가중치와 곱해지고, 결과적으로 가중합으로 계산되어 은닉층 뉴런의 시그모이드 함수의 입력값이 됩니다. 
- z1과 z2는 시그모이드 함수의 입력으로 사용되는 각각의 값에 해당됩니다.

z1 = W1x1 + W2x2 = 0.3 × 0.1 + 0.25 × 0.2 = 0.08  
z2 = W3x1 + W4x2 = 0.4 × 0.1 + 0.35 × 0.2 = 0.11  


- z1과 z2는 각각의 은닉층 뉴런에서 시그모이드 함수를 지나게 되는데 시그모이드 함수가 리턴하는 결과값은 은닉층 뉴런의 최종 출력값입니다. 식에서는 각각 h1과 h2에 해당되며, 아래의 결과와 같습니다.


h1 = sigmoid( z1 ) = 0.51998934  
h2 = sigmoid( z2 ) = 0.52747230  


- h1과 h2 이 두 값은 다시 출력층의 뉴런으로 향하게 되는데 이때 다시 각각의 값에 해당되는 가중치와 곱해지고, 다시 가중합 되어 출력층 뉴런의 시그모이드 함수의 입력값이 됩니다. 식에서는 각각 z3과 z4에 해당됩니다.


z3 = W5h1 + W6h2 = 0.45 × h1 + 0.4 × h2 = 0.44498412  
z4 = W7h1 + W8h2 = 0.7  × h1 + 0.6 × h2 = 0.68047592  


- z3과 z4이 출력층 뉴런에서 시그모이드 함수를 지난 값은 이 인공 신경망이 최종적으로 계산한 출력값입니다. 실제값을 예측하기 위한 값으로서 예측값이라고도 부릅니다.


o1 = sigmoid( z3 ) = 0.60944600  
o2 = sigmoid( z4 ) = 0.66384491  


- 이제 해야할 일은 예측값과 실제값의 오차를 계산하기 위한 오차 함수를 선택하는 것입니다. 오차(Error)를 계산하기 위한 손실 함수(Loss function)로는 평균 제곱 오차 MSE를 사용합니다. 식에서는 실제값을 target이라고 표현하였으며, 순전파를 통해 나온 예측값을 output으로 표현하였습니다. 그리고 각 오차를 모두 더하면 전체 오차 Etotal가 됩니다.

## 3. 역전파 1단계(BackPropagation Step 1)

https://medium.com/@14prakash/back-propagation-is-very-simple-who-made-it-complicated-97b794c97e5c

- 순전파가 입력층에서 출력층으로 향한다면 역전파는 반대로 출력층에서 입력층 방향으로 계산하면서 가중치를 업데이트해갑니다. 
- 출력층 바로 이전의 은닉층을 N층이라고 하였을 때, 출력층과 N층 사이의 가중치를 업데이트하는 단계를 역전파 1단계, 그리고 N층과 N층의 이전층 사이의 가중치를 업데이트 하는 단계를 역전파 2단계라고 해봅시다.

![Alt text]( nn3_final.png )

- 역전파 1단계에서 업데이트 해야 할 가중치는 W5,W6,W7,W8 총 4개입니다. 원리 자체는 동일하므로 우선 W5에 대해서 먼저 업데이트를 진행해보겠습니다. 경사 하강법을 수행하려면 가중치 W5를 업데이트 하기 위해서\begin{equation*}
\frac{∂E_{total}}{∂W_{5}}\end{equation*}를 계산해야 합니다.


- \begin{equation*}\frac{∂E_{total}}{∂W_{5}}\end{equation*}를 계산하기 위해 미분의 연쇄 법칙(Chain rule)에 따라서 이와 같이 풀어 쓸 수 있습니다.

\begin{equation*}
    \frac{∂E_{total}}{∂W_{5}} = \frac{∂E_{total}}{∂o_{1}} \text{×} \frac{∂o_{1}}{∂z_{3}} \text{×} \frac{∂z_{3}}{∂W_{5}}
\end{equation*}

- 위의 식에서 우변의 세 개의 각 항에 대해서 순서대로 계산해봅시다. 우선 첫번째 항에 대해서 계산해보겠습니다. 미분을 진행하기 전에 Etotal의 값을 상기해봅시다. Etotal은 앞서 순전파를 진행하고 계산했던 전체 오차값입니다. 식은 다음과 같습니다.

\begin{equation*}
   E_{total}=\frac{1}{2}(target_{o1}-output_{o1})^{2} + \frac{1}{2}(target_{o2}-output_{o2})^{2}
\end{equation*}

이에 begin{equation*}\\frac{∂E_{total}}{∂o_{1}}\end{equation*}는 다음과 같습니다.

\begin{equation*}
   \frac{∂E_{total}}{∂o_{1}}=2 \text{×} \frac{1}{2}(target_{o1}-output_{o1})^{2-1} \text{×} (-1) + 0
\end{equation*}

\begin{equation*}
   \frac{∂E_{total}}{∂o_{1}}=-(target_{o1}-output_{o1})=-(0.4-0.60944600)=0.20944600
\end{equation*}

- 이제 두번째 항을 주목해봅시다. o1이라는 값은 시그모이드 함수의 출력값입니다. 그런데 시그모이드 함수의 미분은 f(x)×(1−f(x))입니다. 앞으로의 계산 과정에서도 계속해서 시그모이드 함수를 미분해야 하는 상황이 생기므로 기억해둡시다. 이에 따라서 두번째 항의 미분 결과는 다음과 같습니다.
(시그모이드 함수 미분 참고 링크 : https://en.wikipedia.org/wiki/Logistic_function#Derivative)

\begin{equation*}
   \frac{∂o_{1}}{∂z_{3}}=o_{1}\text{×}(1-o_{1})=0.60944600(1-0.60944600)=0.23802157
\end{equation*}



## 4. 역전파 2단계(BackPropagation Step 2)

![Alt text]( nn4.png )

- 1단계를 완료하였다면 이제 입력층 방향으로 이동하며 다시 계산을 이어갑니다. 위의 그림에서 빨간색 화살표는 순전파의 정반대 방향인 역전파의 방향을 보여줍니다. 현재 인공 신경망은 은닉층이 1개밖에 없으므로 이번 단계가 마지막 단계입니다. 하지만 은닉층이 더 많은 경우라면 입력층 방향으로 한 단계씩 계속해서 계산해가야 합니다.

### 5. 결과 확인

![Alt text]( nn1_final.png )

z1 = W1x1 + W2x2 = 0.29959556 × 0.1 + 0.24919112 × 0.2 = 0.07979778  
z2 = W3x1 + W4x2 = 0.39964496 × 0.1 + 0.34928991 × 0.2 = 0.10982248  


h1 = sigmoid( z1 ) = 0.51993887  
h2 = sigmoid( z2 ) = 0.52742806  


z3 = W5h1 + W6h2 = 0.43703857 × h1 + 0.38685205 × h2 = 0.43126996  
z4 = W7h1 + W8h2 = 0.69629578 × h1 + 0.59624247 × h2 = 0.67650625  


o1 = sigmoid( z3 ) = 0.60617688  
o2 = sigmoid( z4 ) = 0.66295848  


Eo1 = 1/2( targeto1 − outputo1 ) 2 = 0.02125445  
Eo2 = 1/2( targeto2 − outputo2 ) 2 = 0.00198189  


Etotal = Eo1 + Eo2 = 0.02323634

- 기존의 전체 오차 Etotal가 0.02397190였으므로 1번의 역전파로 오차가 감소한 것을 확인할 수 있습니다. 인공 신경망의 학습은 오차를 최소화하는 가중치를 찾는 목적으로 순전파와 역전파를 반복하는 것을 말합니다.